In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set style settings
sns.set_theme()
sns.set_palette("husl")

# Notebook-wide settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

def load_data():
    """
    Load both PJM and weather datasets
    """
    # Load PJM data
    pjm_path = '../../data/raw/pjm_dataset/pjm_hourly_est.csv'
    df_pjm = pd.read_csv(pjm_path, parse_dates=['Datetime'])
    
    # Load weather data
    weather_path = '../../data/raw/weather/noaa_data.csv'
    df_weather = pd.read_csv(weather_path, parse_dates=['timestamp'])
    
    return df_pjm, df_weather

# Load datasets
df_pjm, df_weather = load_data()

# Display basic information
print("PJM Dataset Shape:", df_pjm.shape)
print("\nWeather Dataset Shape:", df_weather.shape)

PJM Dataset Shape: (178262, 13)

Weather Dataset Shape: (148993, 31)


In [3]:
def clean_weather_data(df_weather):
    """
    Clean weather data using the validated temperature ranges from EDA
    """
    df = df_weather.copy()
    
    # Define reasonable temperature ranges for each city
    city_limits = {
        'chicago': {'min': -30, 'max': 42},
        'washington': {'min': -26, 'max': 41},
        'pittsburgh': {'min': -28, 'max': 38},
        'columbus': {'min': -27, 'max': 38}
    }
    
    # Filter data within reasonable ranges
    for city, limits in city_limits.items():
        temp_col = f'temperature_{city}'
        df = df[
            (df[temp_col] >= limits['min']) &
            (df[temp_col] <= limits['max'])
        ]
    
    # Add year for filtering
    df['year'] = df['timestamp'].dt.year
    
    # Filter to valid years based on EDA findings
    df = df[(df['year'] < 2005) | (df['year'] >= 2014)]
    
    return df

def prepare_pjm_data(df_pjm):
    """
    Clean and prepare PJM data, focusing on PJME and PJMW
    """
    df = df_pjm.copy()
    
    # Rename datetime column to match weather data
    df.rename(columns={'Datetime': 'timestamp'}, inplace=True)
    
    # Focus on the most complete columns identified in EDA
    columns_to_keep = ['timestamp', 'PJME', 'PJMW']
    df = df[columns_to_keep]
    
    # Forward fill missing values (as these are time series)
    df['PJME'].fillna(method='ffill', inplace=True)
    df['PJMW'].fillna(method='ffill', inplace=True)
    
    return df

def merge_data(df_pjm, df_weather):
    """
    Merge cleaned PJM and weather data
    """
    # Clean individual datasets
    weather_clean = clean_weather_data(df_weather)
    pjm_clean = prepare_pjm_data(df_pjm)
    
    # Merge on timestamp
    df_merged = pd.merge(pjm_clean, weather_clean, on='timestamp', how='inner')
    
    # Sort by timestamp
    df_merged.sort_values('timestamp', inplace=True)
    
    return df_merged

# Execute the cleaning and merging
df_merged = merge_data(df_pjm, df_weather)

# Display information about the merged dataset
print("Merged Dataset Shape:", df_merged.shape)
print("\nDate Range:")
print("Start:", df_merged['timestamp'].min())
print("End:", df_merged['timestamp'].max())
print("\nMissing Values:")
print(df_merged.isnull().sum()[df_merged.isnull().sum() > 0])
print("\nSample of merged data:")
print(df_merged.head())

Merged Dataset Shape: (36000, 34)

Date Range:
Start: 2002-01-01 00:00:00
End: 2018-06-23 02:00:00

Missing Values:
PJME    1
PJMW    1
dtype: int64

Sample of merged data:
               timestamp     PJME    PJMW  avg_wind_speed  precipitation  \
0    2002-01-01 00:00:00      NaN     NaN         15.0000            0.0   
5249 2002-01-01 01:00:00  30393.0  5038.0         14.8375            0.0   
5250 2002-01-01 02:00:00  29265.0  5038.0         14.6750            0.0   
5251 2002-01-01 03:00:00  28357.0  5038.0         14.5125            0.0   
5252 2002-01-01 04:00:00  27899.0  5038.0         14.3500            0.0   

      avg_temperature  max_temperature  min_temperature  temperature  \
0           -8.400000         3.300000       -20.100000   -20.100000   
5249        -8.782181         3.523102       -20.994105   -20.623083   
5250        -9.111362         3.736495       -21.785040   -20.162511   
5251        -9.389349         3.940488       -22.476271   -18.729084   
5252      

In [4]:
def create_temporal_features(df):
    """
    Create temporal features including cyclic encoding for periodic features
    """
    df = df.copy()
    
    # Drop the first row with NaN values
    df = df.dropna()
    
    # Basic temporal features
    df['hour'] = df['timestamp'].dt.hour
    df['day'] = df['timestamp'].dt.day
    df['month'] = df['timestamp'].dt.month
    df['day_of_week'] = df['timestamp'].dt.dayofweek
    df['week_of_year'] = df['timestamp'].dt.isocalendar().week
    
    # Cyclic encoding for periodic features
    # Hour of day
    df['hour_sin'] = np.sin(2 * np.pi * df['hour']/24)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour']/24)
    
    # Day of week
    df['day_of_week_sin'] = np.sin(2 * np.pi * df['day_of_week']/7)
    df['day_of_week_cos'] = np.cos(2 * np.pi * df['day_of_week']/7)
    
    # Month of year
    df['month_sin'] = np.sin(2 * np.pi * df['month']/12)
    df['month_cos'] = np.cos(2 * np.pi * df['month']/12)
    
    # Time of day indicators
    df['is_morning'] = (df['hour'] >= 6) & (df['hour'] < 12)
    df['is_afternoon'] = (df['hour'] >= 12) & (df['hour'] < 18)
    df['is_evening'] = (df['hour'] >= 18) & (df['hour'] < 22)
    df['is_night'] = (df['hour'] >= 22) | (df['hour'] < 6)
    
    # Weekend indicator
    df['is_weekend'] = df['day_of_week'].isin([5, 6])
    
    return df

def create_lag_features(df):
    """
    Create lag features for both energy consumption and weather
    Focusing on patterns relevant for 24h and 7d predictions
    """
    # Hourly lags for past 24 hours
    for i in [1, 2, 3, 6, 12, 24]:
        df[f'PJME_lag_{i}h'] = df['PJME'].shift(i)
        df[f'PJMW_lag_{i}h'] = df['PJMW'].shift(i)
    
    # Daily lags for past week
    for i in [24, 48, 72, 96, 120, 144, 168]:  # 24*7=168 (one week)
        df[f'PJME_lag_{i}h'] = df['PJME'].shift(i)
        df[f'PJMW_lag_{i}h'] = df['PJMW'].shift(i)
    
    # Same hour previous days
    df['PJME_same_hour_1d'] = df['PJME'].shift(24)
    df['PJME_same_hour_7d'] = df['PJME'].shift(168)
    df['PJMW_same_hour_1d'] = df['PJMW'].shift(24)
    df['PJMW_same_hour_7d'] = df['PJMW'].shift(168)
    
    # Rolling means for different windows
    windows = [6, 12, 24, 168]  # 6h, 12h, 24h, 1 week
    for window in windows:
        df[f'PJME_rolling_mean_{window}h'] = df['PJME'].rolling(window=window).mean()
        df[f'PJMW_rolling_mean_{window}h'] = df['PJMW'].rolling(window=window).mean()
        df[f'PJME_rolling_std_{window}h'] = df['PJME'].rolling(window=window).std()
        df[f'PJMW_rolling_std_{window}h'] = df['PJMW'].rolling(window=window).std()
    
    return df

# Create features
df_features = create_temporal_features(df_merged)
df_features = create_lag_features(df_features)

# Display information about the new features
print("Dataset shape after feature creation:", df_features.shape)
print("\nNew temporal features:", [col for col in df_features.columns if col not in df_merged.columns])
print("\nSample of cyclic features:")
print(df_features[['hour', 'hour_sin', 'hour_cos', 'day_of_week', 'day_of_week_sin', 'day_of_week_cos']].head())
print("\nMissing values in lag features:")
lag_cols = [col for col in df_features.columns if 'lag' in col or 'rolling' in col]
print(df_features[lag_cols].isnull().sum().sort_values(ascending=False).head())

Dataset shape after feature creation: (35999, 94)

New temporal features: ['hour', 'day', 'month', 'day_of_week', 'week_of_year', 'hour_sin', 'hour_cos', 'day_of_week_sin', 'day_of_week_cos', 'month_sin', 'month_cos', 'is_morning', 'is_afternoon', 'is_evening', 'is_night', 'is_weekend', 'PJME_lag_1h', 'PJMW_lag_1h', 'PJME_lag_2h', 'PJMW_lag_2h', 'PJME_lag_3h', 'PJMW_lag_3h', 'PJME_lag_6h', 'PJMW_lag_6h', 'PJME_lag_12h', 'PJMW_lag_12h', 'PJME_lag_24h', 'PJMW_lag_24h', 'PJME_lag_48h', 'PJMW_lag_48h', 'PJME_lag_72h', 'PJMW_lag_72h', 'PJME_lag_96h', 'PJMW_lag_96h', 'PJME_lag_120h', 'PJMW_lag_120h', 'PJME_lag_144h', 'PJMW_lag_144h', 'PJME_lag_168h', 'PJMW_lag_168h', 'PJME_same_hour_1d', 'PJME_same_hour_7d', 'PJMW_same_hour_1d', 'PJMW_same_hour_7d', 'PJME_rolling_mean_6h', 'PJMW_rolling_mean_6h', 'PJME_rolling_std_6h', 'PJMW_rolling_std_6h', 'PJME_rolling_mean_12h', 'PJMW_rolling_mean_12h', 'PJME_rolling_std_12h', 'PJMW_rolling_std_12h', 'PJME_rolling_mean_24h', 'PJMW_rolling_mean_24h', 'PJM

In [5]:
def create_weather_features(df):
    """
    Create weather-based features and weather-time interactions
    """
    df = df.copy()
    
    # Average temperature across cities (weighted by population/importance)
    df['temp_avg'] = (
        df['temperature_chicago'] * 0.35 +
        df['temperature_washington'] * 0.25 +
        df['temperature_pittsburgh'] * 0.2 +
        df['temperature_columbus'] * 0.2
    )
    
    # Temperature changes
    for city in ['chicago', 'washington', 'pittsburgh', 'columbus']:
        # Hourly temperature changes
        df[f'temp_change_{city}'] = df[f'temperature_{city}'] - df[f'temperature_{city}'].shift(1)
        
        # Daily temperature changes
        df[f'temp_change_24h_{city}'] = df[f'temperature_{city}'] - df[f'temperature_{city}'].shift(24)
        
        # Temperature extremes
        df[f'temp_extreme_cold_{city}'] = df[f'temperature_{city}'] <= df[f'temperature_{city}'].quantile(0.1)
        df[f'temp_extreme_hot_{city}'] = df[f'temperature_{city}'] >= df[f'temperature_{city}'].quantile(0.9)
    
    # Regional weather features
    df['temp_spread'] = df[[f'temperature_{city}' for city in ['chicago', 'washington', 'pittsburgh', 'columbus']]].max(axis=1) - \
                       df[[f'temperature_{city}' for city in ['chicago', 'washington', 'pittsburgh', 'columbus']]].min(axis=1)
    
    # Average wind speed and precipitation
    df['wind_speed_avg'] = df[[f'avg_wind_speed_{city}' for city in ['chicago', 'washington', 'pittsburgh', 'columbus']]].mean(axis=1)
    df['precipitation_avg'] = df[[f'precipitation_{city}' for city in ['chicago', 'washington', 'pittsburgh', 'columbus']]].mean(axis=1)
    
    # Create weather severity index
    df['weather_severity'] = (
        (df['temp_extreme_cold_chicago'] | df['temp_extreme_hot_chicago']).astype(int) +
        (df['temp_extreme_cold_washington'] | df['temp_extreme_hot_washington']).astype(int) +
        (df['temp_extreme_cold_pittsburgh'] | df['temp_extreme_hot_pittsburgh']).astype(int) +
        (df['temp_extreme_cold_columbus'] | df['temp_extreme_hot_columbus']).astype(int)
    )
    
    return df

def create_interaction_features(df):
    """
    Create interaction features between weather and temporal components
    """
    df = df.copy()
    
    # Temperature-time interactions
    df['temp_hour_sin'] = df['temp_avg'] * df['hour_sin']
    df['temp_hour_cos'] = df['temp_avg'] * df['hour_cos']
    df['temp_month_sin'] = df['temp_avg'] * df['month_sin']
    df['temp_month_cos'] = df['temp_avg'] * df['month_cos']
    
    # Weather severity interactions
    df['severity_weekend'] = df['weather_severity'] * df['is_weekend']
    df['severity_hour'] = df['weather_severity'] * df['hour']
    
    # Time-specific temperature features
    df['temp_morning'] = df['temp_avg'] * df['is_morning']
    df['temp_afternoon'] = df['temp_avg'] * df['is_afternoon']
    df['temp_evening'] = df['temp_avg'] * df['is_evening']
    df['temp_night'] = df['temp_avg'] * df['is_night']
    
    # Temperature change interactions
    df['temp_change_morning'] = df['temp_change_chicago'] * df['is_morning']  # Using Chicago as reference
    df['temp_change_afternoon'] = df['temp_change_chicago'] * df['is_afternoon']
    
    return df

# Create features
df_weather = create_weather_features(df_features)
df_weather = create_interaction_features(df_weather)

# Display information about the new features
print("Dataset shape after weather feature creation:", df_weather.shape)
print("\nNew weather and interaction features:", 
      [col for col in df_weather.columns if col not in df_features.columns])

# Show sample of new features
weather_features = [col for col in df_weather.columns if col not in df_features.columns]
print("\nSample of new weather features:")
print(df_weather[weather_features].head())

# Check for any unexpected missing values
missing_weather = df_weather[weather_features].isnull().sum()
print("\nMissing values in new weather features:")
print(missing_weather[missing_weather > 0])

Dataset shape after weather feature creation: (35999, 127)

New weather and interaction features: ['temp_avg', 'temp_change_chicago', 'temp_change_24h_chicago', 'temp_extreme_cold_chicago', 'temp_extreme_hot_chicago', 'temp_change_washington', 'temp_change_24h_washington', 'temp_extreme_cold_washington', 'temp_extreme_hot_washington', 'temp_change_pittsburgh', 'temp_change_24h_pittsburgh', 'temp_extreme_cold_pittsburgh', 'temp_extreme_hot_pittsburgh', 'temp_change_columbus', 'temp_change_24h_columbus', 'temp_extreme_cold_columbus', 'temp_extreme_hot_columbus', 'temp_spread', 'wind_speed_avg', 'precipitation_avg', 'weather_severity', 'temp_hour_sin', 'temp_hour_cos', 'temp_month_sin', 'temp_month_cos', 'severity_weekend', 'severity_hour', 'temp_morning', 'temp_afternoon', 'temp_evening', 'temp_night', 'temp_change_morning', 'temp_change_afternoon']

Sample of new weather features:
       temp_avg  temp_change_chicago  temp_change_24h_chicago  \
5249 -19.827572                  NaN      

In [6]:
from datetime import datetime, timedelta
import holidays

def create_holiday_features(df):
    """
    Create holiday-related features
    """
    df = df.copy()
    
    # Create US holidays calendar
    us_holidays = holidays.US()
    
    # Basic holiday indicator
    df['is_holiday'] = df['timestamp'].dt.date.map(lambda x: x in us_holidays)
    
    # Create specific holiday season indicators
    def is_christmas_season(date):
        return (date.month == 12 and date.day >= 15) or \
               (date.month == 12 and date.day <= 31) or \
               (date.month == 1 and date.day <= 5)
    
    def is_thanksgiving_season(date):
        return date.month == 11 and date.day >= 20 and date.day <= 30
    
    def is_summer_holiday(date):
        return date.month in [7, 8] or \
               (date.month == 6 and date.day >= 15) or \
               (date.month == 9 and date.day <= 15)
    
    # Apply holiday seasons
    df['is_christmas_season'] = df['timestamp'].map(is_christmas_season)
    df['is_thanksgiving_season'] = df['timestamp'].map(is_thanksgiving_season)
    df['is_summer_holiday'] = df['timestamp'].map(is_summer_holiday)
    
    # Day before/after holiday
    holiday_dates = set(us_holidays)
    df['is_day_before_holiday'] = df['timestamp'].dt.date.map(
        lambda x: (x + timedelta(days=1)) in holiday_dates
    )
    df['is_day_after_holiday'] = df['timestamp'].dt.date.map(
        lambda x: (x - timedelta(days=1)) in holiday_dates
    )
    
    return df

def create_consumption_features(df):
    """
    Create features related to consumption patterns and trends
    """
    df = df.copy()
    
    # Create 24h moving average crossover signals
    df['ma_24h'] = df['PJME'].rolling(window=24).mean()
    df['ma_168h'] = df['PJME'].rolling(window=168).mean()  # 7 days
    df['trend_signal'] = (df['ma_24h'] > df['ma_168h']).astype(int)
    
    # Peak consumption indicators
    df['hour_of_week'] = df['day_of_week'] * 24 + df['hour']
    
    # Calculate peak consumption thresholds
    hourly_peaks = df.groupby('hour')['PJME'].quantile(0.9)
    weekly_hour_peaks = df.groupby('hour_of_week')['PJME'].quantile(0.9)
    
    # Create peak indicators
    df['is_peak_hour'] = df.apply(lambda x: 
        x['PJME'] > hourly_peaks[x['hour']], axis=1)
    df['is_peak_hour_of_week'] = df.apply(lambda x: 
        x['PJME'] > weekly_hour_peaks[x['hour_of_week']], axis=1)
    
    # Add consumption change rates
    df['consumption_change_rate'] = df['PJME'].pct_change()
    df['consumption_change_rate_24h'] = df['PJME'].pct_change(periods=24)
    
    # Add weekly seasonality strength
    df['weekly_seasonality'] = abs(df['PJME'] - df['PJME'].shift(168)) / df['PJME'].shift(168)
    
    return df

# Create final features
df_final = create_holiday_features(df_weather)
df_final = create_consumption_features(df_final)

# Display information about the new features
print("Dataset shape after final feature creation:", df_final.shape)
print("\nNew features:", [col for col in df_final.columns if col not in df_weather.columns])

# Show sample of holiday features
holiday_cols = [col for col in df_final.columns if 'holiday' in col or 'season' in col]
print("\nSample of holiday features:")
print(df_final[holiday_cols].head())

# Show summary of consumption features
consumption_cols = ['trend_signal', 'is_peak_hour', 'is_peak_hour_of_week', 
                   'consumption_change_rate', 'weekly_seasonality']
print("\nSample of consumption features:")
print(df_final[consumption_cols].head())

# Check missing values in new features
new_cols = [col for col in df_final.columns if col not in df_weather.columns]
missing_final = df_final[new_cols].isnull().sum()
print("\nMissing values in new features:")
print(missing_final[missing_final > 0])

Dataset shape after final feature creation: (35999, 142)

New features: ['is_holiday', 'is_christmas_season', 'is_thanksgiving_season', 'is_summer_holiday', 'is_day_before_holiday', 'is_day_after_holiday', 'ma_24h', 'ma_168h', 'trend_signal', 'hour_of_week', 'is_peak_hour', 'is_peak_hour_of_week', 'consumption_change_rate', 'consumption_change_rate_24h', 'weekly_seasonality']

Sample of holiday features:
      is_holiday  is_christmas_season  is_thanksgiving_season  \
5249        True                 True                   False   
5250        True                 True                   False   
5251        True                 True                   False   
5252        True                 True                   False   
5253        True                 True                   False   

      is_summer_holiday  is_day_before_holiday  is_day_after_holiday  \
5249              False                  False                 False   
5250              False                  False           

In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from scipy.stats import spearmanr

def remove_highly_correlated_features(df, threshold=0.95):
    """
    Remove highly correlated features using Spearman correlation
    """
    # Calculate correlation matrix
    correlation_matrix = df.corr(method='spearman')
    
    # Create upper triangle mask
    upper = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))
    
    # Find features with correlation greater than threshold
    to_drop = [column for column in upper.columns if any(upper[column].abs() > threshold)]
    
    print(f"Dropping {len(to_drop)} highly correlated features: {to_drop}")
    
    return df.drop(columns=to_drop)

def prepare_final_features(df, prediction_window='24h'):
    """
    Prepare final feature set for either 24h or 7d prediction
    """
    df = df.copy()
    
    # Handle missing values
    # Forward fill for trend-based features
    trend_cols = ['ma_24h', 'ma_168h', 'trend_signal']
    df[trend_cols] = df[trend_cols].fillna(method='ffill')
    
    # Fill change rates with 0
    change_cols = ['consumption_change_rate', 'consumption_change_rate_24h', 'weekly_seasonality']
    df[change_cols] = df[change_cols].fillna(0)
    
    # Fill temperature changes with 0
    temp_change_cols = [col for col in df.columns if 'temp_change' in col]
    df[temp_change_cols] = df[temp_change_cols].fillna(0)
    
    # Define features to exclude (target variables and timestamps)
    exclude_cols = ['timestamp', 'PJME', 'PJMW']
    
    # Additional exclusions based on prediction window
    if prediction_window == '24h':
        # Exclude weekly features for 24h prediction
        exclude_cols.extend([col for col in df.columns if '168h' in col])
        exclude_cols.extend([col for col in df.columns if '7d' in col])
        min_required_records = 24
    else:  # 7d prediction
        min_required_records = 168
    
    # Remove rows with insufficient history
    df = df.iloc[min_required_records:]
    
    # Get feature columns
    feature_cols = [col for col in df.columns if col not in exclude_cols]
    
    # Remove highly correlated features
    df_features = remove_highly_correlated_features(df[feature_cols])
    
    # Scale features
    scaler = StandardScaler()
    df_scaled = pd.DataFrame(
        scaler.fit_transform(df_features),
        columns=df_features.columns,
        index=df_features.index
    )
    
    # Add back target variables
    df_final = pd.concat([
        df_scaled,
        df[['PJME', 'PJMW']]
    ], axis=1)
    
    return df_final

# Prepare features for both prediction windows
df_24h = prepare_final_features(df_final, prediction_window='24h')
df_7d = prepare_final_features(df_final, prediction_window='7d')

# Display information about final feature sets
print("\n24-hour prediction feature set:")
print("Shape:", df_24h.shape)
print("\nFeature list:")
print(df_24h.columns.tolist())

print("\n7-day prediction feature set:")
print("Shape:", df_7d.shape)
print("\nFeature list:")
print(df_7d.columns.tolist())

# Show sample statistics
print("\nSample statistics for 24h features:")
print(df_24h.describe().round(3))

# Check for any remaining missing values
print("\nMissing values in 24h features:")
print(df_24h.isnull().sum()[df_24h.isnull().sum() > 0])
print("\nMissing values in 7d features:")
print(df_7d.isnull().sum()[df_7d.isnull().sum() > 0])

Dropping 23 highly correlated features: ['max_temperature_chicago', 'min_temperature_chicago', 'max_temperature_washington', 'min_temperature_washington', 'max_temperature_pittsburgh', 'min_temperature_pittsburgh', 'avg_temperature_columbus', 'max_temperature_columbus', 'min_temperature_columbus', 'temperature_columbus', 'week_of_year', 'PJMW_lag_2h', 'PJMW_lag_3h', 'PJME_same_hour_1d', 'PJMW_same_hour_1d', 'PJME_rolling_mean_6h', 'PJMW_rolling_mean_6h', 'PJMW_rolling_mean_24h', 'temp_avg', 'temp_change_columbus', 'temp_change_morning', 'ma_24h', 'hour_of_week']
Dropping 26 highly correlated features: ['max_temperature_chicago', 'min_temperature_chicago', 'max_temperature_washington', 'min_temperature_washington', 'max_temperature_pittsburgh', 'min_temperature_pittsburgh', 'avg_temperature_columbus', 'max_temperature_columbus', 'min_temperature_columbus', 'temperature_columbus', 'week_of_year', 'PJMW_lag_2h', 'PJMW_lag_3h', 'PJME_same_hour_1d', 'PJME_same_hour_7d', 'PJMW_same_hour_1d',

In [8]:
def prepare_and_save_features(df_24h, df_7d):
    """
    Prepare final feature sets and save to disk as CSV
    Note: Change the save paths according to your project structure
    Target variable: PJME (PJM East Region)
    """
    # Add timestamp back to both datasets
    df_24h_final = df_24h.copy()
    df_7d_final = df_7d.copy()
    
    # Define feature sets (excluding PJMW and keeping only necessary columns)
    cols_to_drop = ['PJMW']
    
    df_24h_final = df_24h_final.drop(columns=cols_to_drop)
    df_7d_final = df_7d_final.drop(columns=cols_to_drop)
    
    # Save to CSV format
    # Note: Modify these paths according to your project structure
    save_path_24h = '../../data/processed/features_24h.csv'
    save_path_7d = '../../data/processed/features_7d.csv'
    
    # Save with index (timestamp) included
    df_24h_final.to_csv(save_path_24h, index=True)
    df_7d_final.to_csv(save_path_7d, index=True)
    
    # Print information about saved datasets
    print("Saved feature sets:")
    print(f"24h prediction features shape: {df_24h_final.shape}")
    print(f"7d prediction features shape: {df_7d_final.shape}")
    print("\nFeature sets saved to:")
    print(f"24h features: {save_path_24h}")
    print(f"7d features: {save_path_7d}")
    print("\nNote: Target variable is 'PJME' (PJM East Region)")
    
    # Return the paths for reference
    return save_path_24h, save_path_7d

# Save the feature sets
save_path_24h, save_path_7d = prepare_and_save_features(df_24h, df_7d)

# Verify the saved files
import os
print("\nVerifying saved files:")
print(f"24h features file exists: {os.path.exists(save_path_24h)}")
print(f"7d features file exists: {os.path.exists(save_path_7d)}")

# Display sample of final feature set
print("\nSample of features (24h prediction set):")
pd.read_csv(save_path_24h, nrows=5)

ImportError: Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
A suitable version of pyarrow or fastparquet is required for parquet support.
Trying to import the above resulted in these errors:
 - Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.
 - Missing optional dependency 'fastparquet'. fastparquet is required for parquet support. Use pip or conda to install fastparquet.